# Polars SQL Interface - Comprehensive Workshop

Use SQL syntax with Polars DataFrames through `pl.SQLContext()`.

## What You'll Learn:
- Creating SQL contexts and registering DataFrames
- Basic SQL queries (SELECT, WHERE, ORDER BY)
- Joins with SQL syntax
- Aggregations and GROUP BY
- Subqueries and CTEs (Common Table Expressions)
- Window functions in SQL
- Mixing SQL and Polars expressions
- When to use SQL vs native Polars API

## Why Use SQL with Polars?
- ✅ Familiar syntax for SQL users
- ✅ Great for complex joins and subqueries
- ✅ Easy data exploration
- ✅ Portable queries across databases
- ✅ Still leverages Polars' speed!

## Note:
SQL queries are converted to Polars LazyFrames internally, so you get the same performance!

In [ ]:
import polars as pl
import numpy as np
from datetime import date, datetime, timedelta

print(f"Polars version: {pl.__version__}")

---
# Part 1: Getting Started with SQLContext

## 1.1 Creating a SQL Context

In [ ]:
# Create sample data
df_employees = pl.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'department': ['Sales', 'Engineering', 'Sales', 'Engineering', 'HR'],
    'salary': [50000, 75000, 55000, 80000, 60000],
    'hire_date': [date(2020, 1, 15), date(2019, 3, 10), date(2021, 6, 1), date(2018, 11, 20), date(2022, 2, 14)]
})

print("Employees DataFrame:")
print(df_employees)

In [ ]:
# Method 1: Create SQLContext and register DataFrame explicitly
ctx = pl.SQLContext()
ctx.register("employees", df_employees)

# Execute SQL query
result = ctx.execute("SELECT * FROM employees")

print("Query result:")
print(result.collect())  # SQLContext returns LazyFrame, use .collect()

In [ ]:
# Method 2: Create SQLContext with DataFrames in constructor
ctx = pl.SQLContext(employees=df_employees)

result = ctx.execute("SELECT name, department, salary FROM employees")

print(result.collect())

In [ ]:
# Method 3: Register multiple DataFrames
df_departments = pl.DataFrame({
    'department': ['Sales', 'Engineering', 'HR'],
    'location': ['NYC', 'San Francisco', 'Boston'],
    'budget': [500000, 1000000, 300000]
})

ctx = pl.SQLContext(
    employees=df_employees,
    departments=df_departments
)

print("Registered tables:")
print(ctx.tables())

---
# Part 2: Basic SQL Queries

## 2.1 SELECT and Column Selection

In [ ]:
ctx = pl.SQLContext(employees=df_employees)

# Select all columns
result = ctx.execute("""
    SELECT * FROM employees
""")

print("All columns:")
print(result.collect())

In [ ]:
# Select specific columns
result = ctx.execute("""
    SELECT name, salary FROM employees
""")

print("Specific columns:")
print(result.collect())

In [ ]:
# Column aliases
result = ctx.execute("""
    SELECT 
        name AS employee_name,
        salary AS annual_salary,
        salary / 12 AS monthly_salary
    FROM employees
""")

print("With aliases and calculated columns:")
print(result.collect())

## 2.2 WHERE Clause - Filtering

In [ ]:
# Basic WHERE
result = ctx.execute("""
    SELECT name, department, salary
    FROM employees
    WHERE salary > 60000
""")

print("Salary > 60000:")
print(result.collect())

In [ ]:
# Multiple conditions with AND
result = ctx.execute("""
    SELECT name, department, salary
    FROM employees
    WHERE department = 'Sales' AND salary > 52000
""")

print("Sales dept with salary > 52000:")
print(result.collect())

In [ ]:
# OR conditions
result = ctx.execute("""
    SELECT name, department, salary
    FROM employees
    WHERE department = 'Sales' OR department = 'HR'
""")

print("Sales or HR departments:")
print(result.collect())

In [ ]:
# IN clause
result = ctx.execute("""
    SELECT name, department, salary
    FROM employees
    WHERE department IN ('Sales', 'HR')
""")

print("Using IN clause:")
print(result.collect())

In [ ]:
# BETWEEN
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    WHERE salary BETWEEN 55000 AND 75000
""")

print("Salary between 55k and 75k:")
print(result.collect())

In [ ]:
# LIKE pattern matching
result = ctx.execute("""
    SELECT name, department
    FROM employees
    WHERE name LIKE '%e%'
""")

print("Names containing 'e':")
print(result.collect())

## 2.3 ORDER BY - Sorting

In [ ]:
# Sort ascending
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    ORDER BY salary ASC
""")

print("Sorted by salary (ascending):")
print(result.collect())

In [ ]:
# Sort descending
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    ORDER BY salary DESC
""")

print("Sorted by salary (descending):")
print(result.collect())

In [ ]:
# Multiple sort columns
result = ctx.execute("""
    SELECT name, department, salary
    FROM employees
    ORDER BY department ASC, salary DESC
""")

print("Sorted by department, then salary:")
print(result.collect())

## 2.4 LIMIT and OFFSET

In [ ]:
# LIMIT - Top N results
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    ORDER BY salary DESC
    LIMIT 3
""")

print("Top 3 highest salaries:")
print(result.collect())

In [ ]:
# LIMIT with OFFSET (pagination)
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    ORDER BY salary DESC
    LIMIT 2 OFFSET 2
""")

print("Rows 3-4 (offset 2, limit 2):")
print(result.collect())

---
# Part 3: Aggregations and GROUP BY

## 3.1 Basic Aggregation Functions

In [ ]:
# COUNT, SUM, AVG, MIN, MAX
result = ctx.execute("""
    SELECT 
        COUNT(*) AS total_employees,
        SUM(salary) AS total_salary,
        AVG(salary) AS avg_salary,
        MIN(salary) AS min_salary,
        MAX(salary) AS max_salary
    FROM employees
""")

print("Overall statistics:")
print(result.collect())

## 3.2 GROUP BY

In [ ]:
# Group by department
result = ctx.execute("""
    SELECT 
        department,
        COUNT(*) AS num_employees,
        AVG(salary) AS avg_salary,
        SUM(salary) AS total_salary
    FROM employees
    GROUP BY department
""")

print("Statistics by department:")
print(result.collect())

In [ ]:
# GROUP BY with ORDER BY
result = ctx.execute("""
    SELECT 
        department,
        AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""")

print("Average salary by department (sorted):")
print(result.collect())

## 3.3 HAVING - Filtering Groups

In [ ]:
# HAVING clause (filter after aggregation)
result = ctx.execute("""
    SELECT 
        department,
        COUNT(*) AS num_employees,
        AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
    HAVING COUNT(*) > 1
""")

print("Departments with more than 1 employee:")
print(result.collect())

In [ ]:
# HAVING with multiple conditions
result = ctx.execute("""
    SELECT 
        department,
        COUNT(*) AS num_employees,
        AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
    HAVING COUNT(*) >= 2 AND AVG(salary) > 50000
""")

print("Departments with 2+ employees and avg salary > 50k:")
print(result.collect())

---
# Part 4: Joins

In [ ]:
# Create related tables
df_employees = pl.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'department_id': [1, 2, 1, 2, 3]
})

df_departments = pl.DataFrame({
    'department_id': [1, 2, 3, 4],
    'department_name': ['Sales', 'Engineering', 'HR', 'Marketing'],
    'location': ['NYC', 'San Francisco', 'Boston', 'Chicago']
})

ctx = pl.SQLContext(
    employees=df_employees,
    departments=df_departments
)

print("Employees:")
print(df_employees)
print("\nDepartments:")
print(df_departments)

## 4.1 INNER JOIN

In [ ]:
# INNER JOIN
result = ctx.execute("""
    SELECT 
        e.name,
        d.department_name,
        d.location
    FROM employees e
    INNER JOIN departments d ON e.department_id = d.department_id
""")

print("INNER JOIN (matching records only):")
print(result.collect())

## 4.2 LEFT JOIN

In [ ]:
# LEFT JOIN
result = ctx.execute("""
    SELECT 
        e.name,
        d.department_name,
        d.location
    FROM employees e
    LEFT JOIN departments d ON e.department_id = d.department_id
""")

print("LEFT JOIN (all employees):")
print(result.collect())

## 4.3 RIGHT JOIN

In [ ]:
# RIGHT JOIN
result = ctx.execute("""
    SELECT 
        e.name,
        d.department_name,
        d.location
    FROM employees e
    RIGHT JOIN departments d ON e.department_id = d.department_id
""")

print("RIGHT JOIN (all departments, including Marketing with no employees):")
print(result.collect())

## 4.4 Multiple Joins

In [ ]:
# Add projects table
df_projects = pl.DataFrame({
    'project_id': [101, 102, 103, 104],
    'project_name': ['Website Redesign', 'Mobile App', 'Data Pipeline', 'ML Model'],
    'employee_id': [1, 2, 2, 4]
})

ctx.register('projects', df_projects)

# Join all three tables
result = ctx.execute("""
    SELECT 
        e.name,
        d.department_name,
        p.project_name
    FROM employees e
    INNER JOIN departments d ON e.department_id = d.department_id
    INNER JOIN projects p ON e.employee_id = p.employee_id
""")

print("Three-way join:")
print(result.collect())

---
# Part 5: Subqueries

In [ ]:
# Setup data
df_employees = pl.DataFrame({
    'employee_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'department': ['Sales', 'Engineering', 'Sales', 'Engineering', 'HR'],
    'salary': [50000, 75000, 55000, 80000, 60000]
})

ctx = pl.SQLContext(employees=df_employees)

## 5.1 Subquery in WHERE Clause

In [ ]:
# Find employees with above-average salary
result = ctx.execute("""
    SELECT name, salary
    FROM employees
    WHERE salary > (SELECT AVG(salary) FROM employees)
""")

print("Employees with above-average salary:")
print(result.collect())

## 5.2 Subquery in FROM Clause

In [ ]:
# Use subquery as derived table
result = ctx.execute("""
    SELECT 
        department,
        avg_salary
    FROM (
        SELECT 
            department,
            AVG(salary) AS avg_salary
        FROM employees
        GROUP BY department
    )
    WHERE avg_salary > 60000
""")

print("Departments with average salary > 60000:")
print(result.collect())

## 5.3 Common Table Expressions (CTEs) with WITH

In [ ]:
# CTE - More readable than nested subqueries
result = ctx.execute("""
    WITH dept_stats AS (
        SELECT 
            department,
            AVG(salary) AS avg_salary,
            COUNT(*) AS num_employees
        FROM employees
        GROUP BY department
    )
    SELECT 
        department,
        avg_salary,
        num_employees
    FROM dept_stats
    WHERE num_employees >= 2
    ORDER BY avg_salary DESC
""")

print("Using CTE:")
print(result.collect())

In [ ]:
# Multiple CTEs
result = ctx.execute("""
    WITH 
        high_earners AS (
            SELECT *
            FROM employees
            WHERE salary > 60000
        ),
        dept_counts AS (
            SELECT 
                department,
                COUNT(*) AS num_high_earners
            FROM high_earners
            GROUP BY department
        )
    SELECT *
    FROM dept_counts
    ORDER BY num_high_earners DESC
""")

print("Multiple CTEs:")
print(result.collect())

---
# Part 6: Window Functions

## 6.1 ROW_NUMBER, RANK, DENSE_RANK

In [ ]:
# Window functions for ranking
result = ctx.execute("""
    SELECT 
        name,
        department,
        salary,
        ROW_NUMBER() OVER (ORDER BY salary DESC) AS row_num,
        RANK() OVER (ORDER BY salary DESC) AS rank,
        DENSE_RANK() OVER (ORDER BY salary DESC) AS dense_rank
    FROM employees
""")

print("Window functions for ranking:")
print(result.collect())

## 6.2 PARTITION BY

In [ ]:
# Rank within each department
result = ctx.execute("""
    SELECT 
        name,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) AS dept_rank
    FROM employees
    ORDER BY department, dept_rank
""")

print("Ranking within each department:")
print(result.collect())

## 6.3 Aggregate Window Functions

In [ ]:
# Running totals and averages
result = ctx.execute("""
    SELECT 
        name,
        department,
        salary,
        AVG(salary) OVER (PARTITION BY department) AS dept_avg_salary,
        salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_dept_avg
    FROM employees
    ORDER BY department, name
""")

print("Compare to department average:")
print(result.collect())

---
# Part 7: Advanced SQL Features

## 7.1 CASE WHEN - Conditional Logic

In [ ]:
# CASE statement
result = ctx.execute("""
    SELECT 
        name,
        salary,
        CASE 
            WHEN salary >= 75000 THEN 'High'
            WHEN salary >= 55000 THEN 'Medium'
            ELSE 'Low'
        END AS salary_bracket
    FROM employees
""")

print("Salary brackets:")
print(result.collect())

## 7.2 DISTINCT

In [ ]:
# Get unique departments
result = ctx.execute("""
    SELECT DISTINCT department
    FROM employees
    ORDER BY department
""")

print("Unique departments:")
print(result.collect())

## 7.3 UNION

In [ ]:
# UNION to combine results
result = ctx.execute("""
    SELECT name, 'High Earner' AS category
    FROM employees
    WHERE salary > 70000
    
    UNION ALL
    
    SELECT name, 'Low Earner' AS category
    FROM employees
    WHERE salary < 55000
""")

print("Union of high and low earners:")
print(result.collect())

---
# Part 8: Real-World Examples

## 8.1 Example: Sales Analysis

In [ ]:
# Create sales data
df_sales = pl.DataFrame({
    'order_id': range(1, 11),
    'customer_id': [1, 2, 1, 3, 2, 1, 4, 3, 2, 5],
    'product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Laptop', 'Mouse', 'Keyboard', 'Laptop', 'Monitor', 'Mouse'],
    'amount': [1200, 25, 75, 300, 1200, 25, 75, 1200, 300, 25],
    'order_date': [
        date(2024, 1, 15), date(2024, 1, 16), date(2024, 1, 17),
        date(2024, 1, 18), date(2024, 1, 19), date(2024, 1, 20),
        date(2024, 1, 21), date(2024, 1, 22), date(2024, 1, 23),
        date(2024, 1, 24)
    ]
})

ctx = pl.SQLContext(sales=df_sales)

print("Sales data:")
print(df_sales)

In [ ]:
# Complex analysis with multiple CTEs
result = ctx.execute("""
    WITH 
        -- Customer totals
        customer_totals AS (
            SELECT 
                customer_id,
                COUNT(*) AS num_orders,
                SUM(amount) AS total_spent
            FROM sales
            GROUP BY customer_id
        ),
        -- Product stats
        product_stats AS (
            SELECT 
                product,
                COUNT(*) AS times_sold,
                SUM(amount) AS product_revenue
            FROM sales
            GROUP BY product
        )
    SELECT 
        'Customer' AS entity_type,
        CAST(customer_id AS VARCHAR) AS entity_name,
        total_spent AS revenue
    FROM customer_totals
    WHERE total_spent > 100
    
    UNION ALL
    
    SELECT 
        'Product' AS entity_type,
        product AS entity_name,
        product_revenue AS revenue
    FROM product_stats
    WHERE times_sold >= 3
    
    ORDER BY revenue DESC
""")

print("Top customers and products:")
print(result.collect())

## 8.2 Example: Time Series Analysis

In [ ]:
# Running totals and moving averages
result = ctx.execute("""
    SELECT 
        order_date,
        amount,
        SUM(amount) OVER (
            ORDER BY order_date 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS cumulative_revenue,
        AVG(amount) OVER (
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS moving_avg_3day
    FROM sales
    ORDER BY order_date
""")

print("Time series with cumulative and moving averages:")
print(result.collect())

---
# Part 9: Mixing SQL and Polars Expressions

In [ ]:
# Execute SQL, then continue with Polars methods
lazy_result = ctx.execute("""
    SELECT 
        customer_id,
        SUM(amount) AS total_spent
    FROM sales
    GROUP BY customer_id
""")

# Continue with Polars expressions
final_result = (
    lazy_result
    .filter(pl.col('total_spent') > 500)
    .with_columns([
        (pl.col('total_spent') * 0.1).alias('estimated_profit'),
        pl.when(pl.col('total_spent') > 2000)
          .then(pl.lit('VIP'))
          .otherwise(pl.lit('Regular'))
          .alias('customer_tier')
    ])
    .collect()
)

print("SQL query + Polars expressions:")
print(final_result)

In [ ]:
# Start with Polars, then use SQL
df_filtered = df_sales.filter(pl.col('amount') > 50)

ctx_mixed = pl.SQLContext(filtered_sales=df_filtered)

result = ctx_mixed.execute("""
    SELECT 
        product,
        COUNT(*) AS num_sales,
        AVG(amount) AS avg_amount
    FROM filtered_sales
    GROUP BY product
""")

print("Polars filter + SQL aggregation:")
print(result.collect())

---
# Part 10: SQL vs Native Polars - When to Use Each

## Comparison Table

| Task | SQL | Native Polars | Recommendation |
|------|-----|---------------|----------------|
| **Complex Joins** | ✅ Clear syntax | ❌ Verbose | Use SQL |
| **Subqueries & CTEs** | ✅ Readable | ❌ Harder to nest | Use SQL |
| **Quick exploration** | ✅ Fast to write | ❌ More typing | Use SQL |
| **Method chaining** | ❌ Not possible | ✅ Fluent API | Use Polars |
| **Type safety** | ❌ String-based | ✅ IDE support | Use Polars |
| **Dynamic queries** | ❌ String concat | ✅ Programmatic | Use Polars |
| **Expressions** | ❌ Limited | ✅ Very powerful | Use Polars |
| **For SQL users** | ✅ Familiar | ❌ Learning curve | Use SQL |

## Example: Same Query in Both Styles

In [ ]:
# SQL version
sql_result = ctx.execute("""
    SELECT 
        product,
        COUNT(*) AS num_sales,
        SUM(amount) AS total_revenue,
        AVG(amount) AS avg_sale
    FROM sales
    WHERE amount > 50
    GROUP BY product
    HAVING COUNT(*) >= 2
    ORDER BY total_revenue DESC
""")

print("SQL version:")
print(sql_result.collect())

In [ ]:
# Native Polars version
polars_result = (
    df_sales
    .filter(pl.col('amount') > 50)
    .group_by('product')
    .agg([
        pl.len().alias('num_sales'),
        pl.col('amount').sum().alias('total_revenue'),
        pl.col('amount').mean().alias('avg_sale')
    ])
    .filter(pl.col('num_sales') >= 2)
    .sort('total_revenue', descending=True)
)

print("Native Polars version:")
print(polars_result)

---
# Summary

## Key Takeaways:

### 1. **SQLContext Basics**
   - Create with `pl.SQLContext()`
   - Register DataFrames with `ctx.register()` or in constructor
   - Execute queries with `ctx.execute()`
   - Returns LazyFrame - use `.collect()` to materialize

### 2. **Supported SQL Features**
   - ✅ SELECT, WHERE, GROUP BY, HAVING, ORDER BY
   - ✅ JOINS (INNER, LEFT, RIGHT)
   - ✅ Subqueries and CTEs (WITH)
   - ✅ Window functions (ROW_NUMBER, RANK, etc.)
   - ✅ Aggregations (COUNT, SUM, AVG, MIN, MAX)
   - ✅ CASE WHEN, DISTINCT, UNION

### 3. **Best Practices**
   - Use SQL for complex joins and readable queries
   - Use native Polars for method chaining and type safety
   - Mix both: SQL for exploration, Polars for production
   - CTEs make complex queries more readable
   - Leverage window functions for analytics

### 4. **Common Patterns**

```python
# Create context
ctx = pl.SQLContext(table_name=df)

# Execute query
result = ctx.execute("SELECT * FROM table_name")

# Materialize
df_result = result.collect()

# Mix with Polars
final = result.filter(pl.col('x') > 10).collect()
```

### 5. **When to Use SQL**
   - ✅ You already know SQL well
   - ✅ Complex multi-table joins
   - ✅ Quick data exploration
   - ✅ Porting queries from other databases
   - ❌ Need dynamic query building
   - ❌ Want IDE autocomplete and type safety

## Performance Note:
> SQL queries in Polars are converted to LazyFrame operations internally, so performance is **identical** to native Polars expressions!

## Remember:
> Use the approach that makes your code **most readable** and **maintainable** for your use case. Polars SQL gives you the best of both worlds!

---
# Practice Exercises

In [ ]:
# Exercise data
df_orders = pl.DataFrame({
    'order_id': range(1, 11),
    'customer_id': [101, 102, 101, 103, 102, 104, 101, 103, 105, 102],
    'product': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C'],
    'quantity': [2, 1, 3, 1, 2, 1, 2, 3, 1, 2],
    'price': [100, 200, 100, 150, 200, 100, 150, 200, 100, 150],
    'order_date': [
        date(2024, 1, i) for i in range(1, 11)
    ]
})

ctx_exercise = pl.SQLContext(orders=df_orders)
print("Exercise data:")
print(df_orders)

In [ ]:
# Exercise 1: Find total revenue (quantity * price) by product
# Your SQL here:


In [ ]:
# Exercise 2: Find customers who ordered more than 2 times
# Your SQL here:


In [ ]:
# Exercise 3: Calculate running total of revenue over time
# Hint: Use window function with ORDER BY order_date
# Your SQL here:


In [ ]:
# Exercise 4: Using CTE, find products with above-average revenue
# Your SQL here:


In [ ]:
# Exercise 5: Rank customers by total spending within each product category
# Your SQL here:
